In [1]:
# 이 책에서 다루는 음성 데이터와 라벨 데이터를 다운로드
# 데이터는 JSUT 코퍼스를 사용
# https://sites.google.com/site/shinnosuketakamichi/publication/jsut

# 파일 다운로드
from urllib.request import urlretrieve

# zip 파일
import zipfile

# os
import os

# 메인 함수
if __name__ == "__main__":
  # 데이터 저장 위치
  data_dir = 'Users/juyoung/speech_recognition/data/original'

  # 디렉토리 data_dir가 존재하지 않으면 작성
  os.makedirs(data_dir, exist_ok=True)

  # 음성 파일(jsut 코퍼스.zip형식) 다운로드
  data_archive = os.path.join(data_dir, 'jsut-data.zip')
  print('download jsut-data start')
  urlretrieve('http://ss-takashi.sakura.ne.jp/corpus/jsut_ver1.1.zip',
              data_archive)
  print('download jsut-data finished')

  # 다운로드한 zip 데이터 압축해제
  print('extract jsut-data start')
  with zipfile.ZipFile(data_archive) as data_zip:
    data_zip.extractall(data_dir)
  print('extract jsut-data finished')

  # zip 파일을 삭제
  os.remove(data_archive)

  # jsut 코퍼스 라벨 데이터 다운로드
  label_archive = os.path.join(data_dir, 'jsut-label.zip')
  print('download jsut-label start')
  urlretrieve('https://github.com/sarulab-speech/jsut-label/archive/master.zip',
              label_archive)
  print('download jsut-label finished')

  print('extract jsut-label start')
  with zipfile.ZipFile(label_archive) as label_zip:
    label_zip.extractall(data_dir)
  print('extract jsut-label finished')

  os.remove(label_archive)

  print('all processes finished')


download jsut-data start
download jsut-data finished
extract jsut-data start
extract jsut-data finished
download jsut-label start
download jsut-label finished
extract jsut-label start
extract jsut-label finished
all processes finished


In [2]:
# 다운로드한 wav 파일 48kHZ을 샘플링 레이트 16kHz의 데이터로 변환
# 변환한 wav 데이터의 리스트를 작성
!pip install sox
# 샘플링 주파수를 변환하기 위해 sox사용
# This install of SoX cannot process .wav files. 에러 발생
# mac os는 지원을 안하는 듯, 다른 라이브러리 찾을 때까지 다운 샘플링 처리하지 않고 원본 그대로 사용한다

import sox

# os
import os

# 메인 함수
if __name__ == "__main__":

  # wav 파일이 전개된 디렉토리
  original_wav_dir = 'Users/juyoung/speech_recognition/data/original/jsut_ver1.1/basic5000/wav'

  # 포맷 변환한 wav파일을 출력하는 디렉토리
  out_wav_dir = 'Users/juyoung/speech_recognition/data/wav'

  # wav 데이터 라벨을 저장하는 디렉토리
  out_scp_dir = 'Users/juyoung/speech_recognition/data/label/all'

  # 출력 디렉토리가 없는 경우 작성
  os.makedirs(out_wav_dir, exist_ok=True)
  os.makedirs(out_scp_dir, exist_ok=True)

  # sox에 의한 음성 변환 클래스를 호출
  tfm = sox.Transformer()
  # 샘플링 주파수를 16000Hz로 변환하도록 설정
  tfm.convert(samplerate=16000)

  # wav 데이터의 리스트 파일을 기입 모드로 열고, 이후의 처리 실시
  with open(os.path.join(out_scp_dir, 'wav.scp'), mode='w') as scp_file:
    # BASIC5000_0001.wav ~ BASIC5000_5000.wav에 대해 처리 반복 실행
    for i in range(5000):
      filename = 'BASIC5000_%04d' % (i + 1)
      # 변환원의 원본 데이터(48000Hz)의 파일명
      wav_path_in = os.path.join(original_wav_dir, filename + '.wav')
      # 변환 후 데이터(16000Hz) 저장 파일명
      wav_path_out = os.path.join(out_wav_dir, filename + '.wav')

      print(wav_path_in)
      # 파일이 존재하지 않는 경우 오류
      if not os.path.exists(wav_path_in):
        print('Error: Not found %s' % (wav_path_in))
        exit()

      # 샘플링 주파수 변환 및 저장을 실행
      tfm.build_file(input_filepath=wav_path_in,
                     output_filepath=wav_path_out)

      # wav 파일 스크립트
      scp_file.write('%s %s\n' %
                     (filename, os.path.abspath(wav_path_out)))


/bin/sh: sox: command not found
SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    
This install of SoX cannot process .wav files.
This install of SoX cannot process .wav files.


Users/juyoung/speech_recognition/data/original/jsut_ver1.1/basic5000/wav/BASIC5000_0001.wav


FileNotFoundError: [Errno 2] No such file or directory: 'sox'

In [3]:
# 다운로드한 명찰 데이터를 읽기，
# 문자(한자 섞인) 단위, 히라가나 단위, 음소 단위로 정의되는 라벨 파일을 작성

# yaml 데이터 가져오기
import yaml

# os
import os

# 메인 함수
if __name__ == "__main__":

  # 다운로드한 라벨데이터(yaml형식)
  original_label = 'Users/juyoung/speech_recognition/data/original/jsut-label-master/text_kana/basic5000.yaml'

  # 라벨 리스트를 격납하는 장소
  out_label_dir = 'Users/juyoung/speech_recognition/data/label/all'

  # 출력 디렉터리가 없는 경우 작성
  os.makedirs(out_label_dir, exist_ok=True)

  # 라벨 데이터 열기
  # 인코딩 오류 해결을 위해 'UTF-8' 사용
  with open(original_label, mode='r', encoding='UTF-8') as yamlfile:
    label_info = yaml.safe_load(yamlfile)

  # 문자/히라가나/음소의 라벨 파일을 각각 저장
  # 인코딩 오류 해결을 위해 'UTF-8' 사용
  with open(os.path.join(out_label_dir, 'text_char'),
            mode='w', encoding='UTF-8') as label_char, \
      open(os.path.join(out_label_dir, 'text_kana'),
           mode='w', encoding='UTF-8') as label_kana, \
      open(os.path.join(out_label_dir, 'text_phone'),
           mode='w', encoding='UTF-8') as label_phone:
    # BASIC5000_0001 ~ BASIC5000_5000 까지 반복 실행
    for i in range(5000):
      # 발화ID
      filename = 'BASIC5000_%04d' % (i + 1)

      # 발화ID가 label_info에 포함되지 않을 경우 에러
      if not filename in label_info:
        print('Error: %s is not in %s' % (filename, original_label))
        exit()

      # 문자 라벨 정보
      chars = label_info[filename]['text_level2']
      # '、'와 '。'를 제거
      chars = chars.replace('、', '')
      chars = chars.replace('。', '')

      # 히라가나 라벨 정보
      kanas = label_info[filename]['kana_level3']
      # '、'를 제거
      kanas = kanas.replace('、', '')

      # 음소 라벨 정보
      phones = label_info[filename]['phone_level3']

      # 문자 라벨 파일에 한 글자씩 뛰어서 공백으로 구분
      # ('.join(list)은 리스트의 각 요소에 스페이스를 사이에 두고, 1문장으로 한다)
      label_char.write('%s %s\n' % (filename, ' '.join(chars)))

      # 히라가나 라벨 파일에 한 글자씩 뛰어서 공백으로 구분
      label_kana.write('%s %s\n' % (filename, ' '.join(kanas)))

      # 음소 라벨은 '-'를 스페이스로 치환해서 표시
      label_phone.write('%s %s\n' % (filename, phones.replace('-', ' ')))

In [4]:
# 데이터 리스트를, 학습/개발/평가 데이터 세트로 분할
# BASIC5000_0001~0250 : 평가 데이터
# BASIC5000_0251~0500 : 개발 데이터
# BASIC5000_0501~1500 : 학습 데이터(소)
# BASIC5000_0501~5000 : 학습 데이터(대）

# os
import os

# 메인 함수
if __name__ == "__main__":

  # 전체 데이터의 저장 위치
  all_dir = 'Users/juyoung/speech_recognition/data/label/all'

  # 평가 데이터 출력
  out_eval_dir = 'Users/juyoung/speech_recognition/data/label/test'
  # 개발 데이터 출력
  out_dev_dir = 'Users/juyoung/speech_recognition/data/label/dev'
  # 학습 데이터(소) 출력
  out_train_small_dir = 'Users/juyoung/speech_recognition/data/label/train_small'
  # 학습 데이터(대) 출력
  out_train_large_dir = 'Users/juyoung/speech_recognition/data/label/train_large'

  # 각 출력 디렉토리가 존재하지 않는 경우 작성
  for out_dir in [out_eval_dir, out_dev_dir,
                  out_train_small_dir, out_train_large_dir]:
    os.makedirs(out_dir, exist_ok=True)

  # wav.scp, text_char, text_kana, text_phone 각각 동일하게 처리
  for filename in ['wav.scp', 'text_char',
                   'text_kana', 'text_phone']:
    # 평가/개발/학습 데이터 리스트를 각각 저장
    # 인코딩 오류 해결을 위해 'UTF-8' 사용
    with open(os.path.join(all_dir, filename),
              mode='r', encoding='UTF-8') as all_file, \
        open(os.path.join(out_eval_dir, filename),
             mode='w', encoding='UTF-8') as eval_file, \
        open(os.path.join(out_dev_dir, filename),
             mode='w', encoding='UTF-8') as dev_file, \
        open(os.path.join(out_train_small_dir, filename),
             mode='w', encoding='UTF-8') as train_small_file, \
        open(os.path.join(out_train_large_dir, filename),
             mode='w', encoding='UTF-8') as train_large_file:
      # 평가/개발/학습 데이터 리스트에 추가
      for i, line in enumerate(all_file):
        if i < 250:
          # 1~250: 평가 데이터
          eval_file.write(line)
        elif i < 500:
          # 251~500: 개발 데이터
          dev_file.write(line)
        else:
          # 501~5000: 학습 데이터(소)
          train_large_file.write(line)
          if i < 1500:
            # 501～1500: 학습 데이터(대)
            train_small_file.write(line)